In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
SHEET_NAME_STOCK_INFO = "stock_info"
SHEET_NAME_STOCK_FINANCIALS = "stock_financials"
SHEET_NAME_STOCK_DIVIDENDS = "stock_dividends"

# Data Cleansing

In [6]:
stock_info = pd.read_excel("stock_info.xlsx", sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")

In [8]:
len(stock_info), len(stock_info["symbol"].unique())

(6241, 6164)

In [16]:
len(stock_info[stock_info["symbol"].isna()])

78

In [18]:
df = stock_info.dropna(subset=["symbol"])
len(df)

6163

In [20]:
df.head(10)

,52WeekChange,SandP52WeekChange,address1,algorithm,annualHoldingsTurnover,annualReportExpenseRatio,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,beta3Year,bid,bidSize,bookValue,category,circulatingSupply,city,coinMarketCapLink,companyOfficers,country,currency,currentPrice,currentRatio,dateShortInterest,dayHigh,dayLow,debtToEquity,dividendRate,dividendYield,earningsGrowth,earningsQuarterlyGrowth,ebitda,ebitdaMargins,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,exDividendDate,exchange,exchangeTimezoneName,exchangeTimezoneShortName,expireDate,fax,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,financialCurrency,fiveYearAverageReturn,fiveYearAvgDividendYield,floatShares,forwardEps,forwardPE,freeCashflow,fromCurrency,fullTimeEmployees,fundFamily,fundInceptionDate,gmtOffSetMilliseconds,grossMargins,grossProfits,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,industry,isEsgPopulated,lastCapGain,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastMarket,lastSplitDate,lastSplitFactor,legalType,logo_url,longBusinessSummary,longName,market,marketCap,maxAge,maxSupply,messageBoardId,morningStarOverallRating,morningStarRiskRating,mostRecentQuarter,navPrice,netIncomeToCommon,nextFiscalYearEnd,numberOfAnalystOpinions,open,openInterest,operatingCashflow,operatingMargins,payoutRatio,pegRatio,phone,preMarketPrice,previousClose,priceHint,priceToBook,priceToSalesTrailing12Months,profitMargins,quickRatio,quoteType,recommendationKey,recommendationMean,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketPrice,regularMarketVolume,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,revenueQuarterlyGrowth,sector,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortName,shortPercentOfFloat,shortRatio,startDate,state,strikePrice,symbol,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,threeYearAverageReturn,toCurrency,totalAssets,totalCash,totalCashPerShare,totalDebt,totalRevenue,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingEps,trailingPE,trailingPegRatio,twoHundredDayAverage,volume,volume24Hr,volumeAllCurrencies,website,yield,ytdReturn,zip
0,-0.241917,-0.111159,5301 Stevens Creek Boulevard,NaN,NaN,NaN,129.88,1000.0,1702650.0,1502291.0,1702650.0,1.067119,NaN,129.60,800.0,17.188,NaN,NaN,Santa Clara,NaN,[],United States,USD,129.8900,1.940,1.661904e+09,132.1300,128.2900,60.361,0.84,0.0065,0.279,0.246,1.922000e+09,0.28863,21.045,6.074,4.044871e+10,1.656634e+09,NYQ,America/New_York,EDT,NaN,866 497 1134,129.809800,174.350000,112.520000,USD,NaN,0.73,2.950903e+08,5.50,23.616364,829000000.0,NaN,17700.0,NaN,NaN,-14400000.0,0.54212,3.407000e+09,0.00239,0.93772,0.0,Diagnostics & Research,0.0,NaN,1.656634e+09,0.210000,1.635638e+09,NaN,1.414973e+09,1398:1000,NaN,https://logo.clearbit.com/agilent.com,"Agilent Technologies, Inc. provides applicatio...","Agilent Technologies, Inc.",us_market,3.845276e+10,1.0,NaN,finmb_154924,NaN,NaN,1.659226e+09,NaN,1.328000e+09,1.698710e+09,NaN,132.1200,NaN,1.305000e+09,0.23938,0.1881,2.31,800 227 9770,NaN,133.1600,2.0,7.557016,5.774556,0.19943,1.273,EQUITY,none,NaN,132.1300,128.2900,132.1200,133.1600,129.8900,2154507.0,0.09499,0.26462,0.083,22.178,NaN,Healthcare,2.960410e+08,0.0091,2705705.0,1.659053e+09,3011237.0,"Agilent Technologies, Inc.",0.0092,1.94,NaN,CA,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.077000e+09,3.638,3.073000e+09,6.659000e+09,0.0,0.824,0.006188,4.380,29.65525,2.2062,132.666700,2154507.0,NaN,NaN,https://www.agilent.com,NaN,NaN,95051
1,-0.123813,-0.111159,201 Isabella Street,NaN,NaN,NaN,42.65,1200.0,7148020.0,5903343.0,7148020.0,2.318817,NaN,42.50,900.0,31.491,NaN,NaN,Pittsburgh,NaN,[],United States,USD,42.4600,1.812,1.661904e+09,43.7600,41.6100,24.863,0.40,0.0094,0.810,0.777,3.546000e+09,0.26490,2.662,0.705,9.440624e+09,1.659917e+09,NYQ,America/New_York,EDT,NaN,NaN,48.771800,98.090000,39.565000,USD,NaN,NaN,1.

# Data Cleansing

In [6]:
FROM_FILE = "data/alphalib_united_states_6.xlsx"
TO_FILE = "data/alphalib_united_states_new.xlsx"

stock_info_1 = pd.read_excel(TO_FILE, sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")
stock_info_2 = pd.read_excel(FROM_FILE, sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")
print(len(stock_info_1.columns), len(stock_info_2.columns))

156 154


In [7]:
def create_missing_cols(df, target_cols):
    columns = df.columns.tolist()
    missing_cols = list(set(target_cols) - set(columns))
    df[missing_cols] = None

In [8]:
# Create missing columns for stock_info
print("Create missing columns")
if len(stock_info_1.columns) > len(stock_info_2.columns):
    stock_info_columns = stock_info_1.columns.tolist()
    stock_info_columns.sort()
    create_missing_cols(stock_info_2, stock_info_columns)
    stock_info_2 = stock_info_2[stock_info_columns]
    print(len(stock_info_1.columns), len(stock_info_2.columns))
elif len(stock_info_1.columns) < len(stock_info_2.columns):
    stock_info_columns = stock_info_2.columns.tolist()
    stock_info_columns.sort()
    create_missing_cols(stock_info_1, stock_info_columns)
    stock_info_1 = stock_info_1[stock_info_columns]
    print(len(stock_info_1.columns), len(stock_info_2.columns))
else:
    print("No action required")
    print(len(stock_info_1.columns), len(stock_info_2.columns))
    
# Remove overlapped stocks
print("Remove overlapped stocks")
filter = ~stock_info_2["symbol"].isin(stock_info_1["symbol"].tolist()) 
print(len(stock_info_2),len(stock_info_2[filter]))
stock_info_2 = stock_info_2[filter]

Create missing columns
156 156
Remove overlapped stocks
223 216


In [9]:
# Merge the 2 data frames
stock_info_all = pd.concat([stock_info_1, stock_info_2], ignore_index=True)
print(len(stock_info_all))

3786


In [10]:
# Clean up stock_dividends
stock_dividends_1 = pd.read_excel(TO_FILE, sheet_name=SHEET_NAME_STOCK_DIVIDENDS, engine="openpyxl")
stock_dividends_2 = pd.read_excel(FROM_FILE, sheet_name=SHEET_NAME_STOCK_DIVIDENDS, engine="openpyxl")
print(len(stock_dividends_1.columns), len(stock_dividends_2.columns))

print(stock_dividends_1.columns)
print(stock_dividends_2.columns)

cols_1 = stock_dividends_1.columns.tolist()
cols_1.sort()

cols_2 = stock_dividends_2.columns.tolist()
cols_2.sort()

if (cols_1 == cols_2):
    print("Same columns")
else:
    print("!!!!!!!!!!! Not the same ")
    if len(stock_dividends_1.columns) > len(stock_dividends_2.columns):
        stock_dividends_columns = stock_dividends_1.columns.tolist()
        stock_dividends_columns.sort()
        create_missing_cols(stock_dividends_2, stock_dividends_columns)
        stock_dividends_2 = stock_dividends_2[stock_dividends_columns]
        print(len(stock_dividends_1.columns), len(stock_dividends_2.columns))
    elif len(stock_dividends_1.columns) < len(stock_dividends_2.columns):
        stock_dividends_columns = stock_dividends_2.columns.tolist()
        stock_dividends_columns.sort()
        create_missing_cols(stock_dividends_1, stock_dividends_columns)
        stock_dividends_1 = stock_dividends_1[stock_dividends_columns]
        print(len(stock_dividends_1.columns), len(stock_dividends_2.columns))

9 6
Index(['Country', 'Date', 'Dividend', 'Full Name', 'Name', 'Payment Date',
       'Symbol', 'Type', 'Yield'],
      dtype='object')
Index(['Country', 'Date', 'Dividends', 'Full Name', 'Name', 'Symbol'], dtype='object')
!!!!!!!!!!! Not the same 
9 9


In [11]:
print("Remove overlapped stocks")
filter = ~stock_dividends_2["Symbol"].isin(stock_dividends_1["Symbol"].tolist()) 
print(len(stock_dividends_2), len(stock_dividends_2[filter]))
stock_dividends_2 = stock_dividends_2[filter]

# Merge the 2 data frames
stock_dividends_all = pd.concat([stock_dividends_1, stock_dividends_2], ignore_index=True)
print(len(stock_dividends_all))
print(len(stock_dividends_1), len(stock_dividends_2))

Remove overlapped stocks
3770 3728
56043
52315 3728


In [12]:
# Clean up stock_financials
stock_financials_1 = pd.read_excel(TO_FILE, sheet_name=SHEET_NAME_STOCK_FINANCIALS, engine="openpyxl")
stock_financials_2 = pd.read_excel(FROM_FILE, sheet_name=SHEET_NAME_STOCK_FINANCIALS, engine="openpyxl")
print(len(stock_financials_1.columns), len(stock_financials_2.columns))

cols_1 = stock_financials_1.columns.tolist()
cols_1.sort()

cols_2 = stock_financials_2.columns.tolist()
cols_2.sort()

if (cols_1 == cols_2):
    print("Same columns")
    print(cols_1)
else:
    print("!!!!!!!!!!! Not the same ")
    if len(stock_financials_1.columns) > len(stock_financials_2.columns):
        stock_financials_columns = stock_financials_1.columns.tolist()
        stock_financials_columns.sort()
        create_missing_cols(stock_financials_2, stock_financials_columns)
        stock_financials_2 = stock_financials_2[stock_financials_columns]
        print(len(stock_financials_1.columns), len(stock_financials_2.columns))
    elif len(stock_financials_1.columns) < len(stock_financials_2.columns):
        stock_financials_columns = stock_financials_2.columns.tolist()
        stock_financials_columns.sort()
        create_missing_cols(stock_financials_1, stock_financials_columns)
        stock_financials_1 = stock_financials_1[stock_financials_columns]
        print(len(stock_financials_1.columns), len(stock_financials_2.columns))

27 27
Same columns
['Cost Of Revenue', 'Country', 'Date', 'Discontinued Operations', 'Ebit', 'Effect Of Accounting Charges', 'Extraordinary Items', 'Full Name', 'Gross Profit', 'Income Before Tax', 'Income Tax Expense', 'Interest Expense', 'Minority Interest', 'Name', 'Net Income', 'Net Income Applicable To Common Shares', 'Net Income From Continuing Ops', 'Non Recurring', 'Operating Income', 'Other Items', 'Other Operating Expenses', 'Research Development', 'Selling General Administrative', 'Symbol', 'Total Operating Expenses', 'Total Other Income Expense Net', 'Total Revenue']


In [13]:
print("Remove overlapped stocks")
filter = ~stock_financials_2["Symbol"].isin(stock_financials_1["Symbol"].tolist()) 
print(len(stock_financials_2), len(stock_financials_2[filter]))
stock_financials_2 = stock_financials_2[filter]

# Merge the 2 data frames
stock_financials_all = pd.concat([stock_financials_1, stock_financials_2], ignore_index=True)
print(len(stock_financials_all))
print(len(stock_financials_1), len(stock_financials_2))

Remove overlapped stocks
903 875
15208
14333 875


In [14]:
# Save to new file
with pd.ExcelWriter(TO_FILE) as writer:  
    stock_info_all.to_excel(writer,sheet_name=SHEET_NAME_STOCK_INFO, header=True, index=False)
    stock_dividends_all.to_excel(writer,sheet_name=SHEET_NAME_STOCK_DIVIDENDS, header=True, index=False)
    stock_financials_all.to_excel(writer,sheet_name=SHEET_NAME_STOCK_FINANCIALS, header=True, index=False)

In [15]:
# validate the data
stock_info_new = pd.read_excel(TO_FILE, sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")
print(len(stock_info_new))

3786
